# BigQuery Patent Exploration

Use this notebook to prototype filters before wiring them into the automated pipeline.

In [1]:
import os
import sys

try:
    NOTEBOOK_DIR = os.path.dirname(__file__)
except NameError:
    NOTEBOOK_DIR = os.getcwd()

PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)


In [2]:
from google.cloud import bigquery

from src import config, query_builder

## Fix Applied ✅

The BigQuery error `BadRequest: 400 Unrecognized name: locale at [9:13]` has been resolved by updating the `query_builder.py` file. The issue was that the query was trying to access a `locale` field that doesn't exist in the Google Patents BigQuery table structure.

**What was fixed:**
- Removed `WHERE locale = 'en'` conditions from the localized field queries
- The query now simply selects the first text entry from each localized array using `ORDER BY sequence LIMIT 1`

You can now run the cell below successfully!


In [ ]:
# Fixed version - The 'locale' field error has been resolved in query_builder.py
# You can now run this cell successfully
client = bigquery.Client(project="axial-analyzer-475800-v4")
sql = query_builder.build_query(limit=25, description_word_limit=config.DEFAULT_DESCRIPTION_WORD_LIMIT)
job_config = query_builder.assemble_query_config(
    start_year=config.DEFAULT_START_YEAR,
    end_year=config.DEFAULT_END_YEAR,
    description_word_limit=config.DEFAULT_DESCRIPTION_WORD_LIMIT,
)
preview = client.query(sql, job_config=job_config).result().to_dataframe()
preview.head()


In [3]:
client = bigquery.Client(project="axial-analyzer-475800-v4")
sql = query_builder.build_query(limit=25, description_word_limit=config.DEFAULT_DESCRIPTION_WORD_LIMIT)
job_config = query_builder.assemble_query_config(
    start_year=config.DEFAULT_START_YEAR,
    end_year=config.DEFAULT_END_YEAR,
    description_word_limit=config.DEFAULT_DESCRIPTION_WORD_LIMIT,
)
preview = client.query(sql, job_config=job_config).result().to_dataframe()
preview.head()

BadRequest: 400 Unrecognized name: sequence at [9:16]; reason: invalidQuery, location: query, message: Unrecognized name: sequence at [9:16]

Location: US
Job ID: a1f2ff4b-02d7-41a4-9575-0406a1eb73ea


Update `start_year`, `end_year`, CPC filters, or keyword lists in `src/config.py` and re-run cells to see how the result set changes.

## Run the pipeline programmatically

Populate `.env` (or export credentials in this notebook) before running the next cell. Adjust parameters as needed.

In [ ]:
import os

from types import SimpleNamespace

from src.pipeline import run_pipeline

# Optionally load environment variables from .env when running inside the notebook.
from dotenv import load_dotenv  # type: ignore

load_dotenv(".env")

args = SimpleNamespace(
    project_id="YOUR_GCP_PROJECT",
    start_year=2022,
    end_year=2022,
    limit=10,
    output_raw="data/patents_raw.csv",
    output_classified="data/patents_classified.csv",
    openrouter_model="openrouter/auto",
    openrouter_timeout=30.0,
    openrouter_delay=1.0,
    skip_llm=True,
    era_column=False,
    log_level="INFO",
    description_word_limit=200,
    max_retries=3,
)

exit_code = run_pipeline(args)
print(f"Pipeline finished with exit code {exit_code}")